In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from geopy import distance

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Import json packages to explode json columns like related_same_day_brand
import json
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


In [ ]:
#download the data
# !wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
%%time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

CPU times: user 2.62 s, sys: 4 ms, total: 2.63 s
Wall time: 2.62 s


In [136]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.8947051766,-73.8472005205
1,Bronx,Co-op City,40.8742941930,-73.8299391081
2,Bronx,Eastchester,40.8875556774,-73.8278064472
3,Bronx,Fieldston,40.8954374269,-73.9056425959
4,Bronx,Riverdale,40.8908344939,-73.9125854611


In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

In [ ]:
%%time
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [ ]:
# CLIENT_ID = 'U54SVMROBPRP3B0NW1FF5QPCCEPMKZS0PHHUDNUTQ3KBNFX2' # your Foursquare ID
# CLIENT_SECRET = 'Y5302JRTVUIGCWR1EROFFHUIGFFFAD50N2AZ4J4ND3JN1TSK' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version

# AR-data-science auths
CLIENT_ID = 'RU3Y3XCL1D3X4IFWHEPI3VYYAEEGSWMVQTWP2PHZ1DEL1E2R' # your Foursquare ID
CLIENT_SECRET = 'GMMPCVSMWDJDXSOTEO22G3FN4H2BUFSZO05SYEGRGW4N5AKL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# type your answer here
LIMIT = 100
radius = 500


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        try:
            response = requests.get(url).json()
            results = response['response']['groups'][0]['items']
        except Exception as ex:
            print("{}, problem here {}".format(name, response['meta']['errorDetail']))
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
%%time
nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [6]:
nyc_venues = pd.read_csv('nyc_venues.csv')
print(nyc_venues.shape)
# Foursquare may respond: Quota exceeded, then I have the content....
# other minor problem: Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.
# nyc_venues.to_csv(path_or_buf='nyc_venues.csv')
nyc_venues.head(50)
# last response (10343, 7)

(20656, 8)


,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
5,5,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place
6,6,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
7,7,Wakefield,40.894705,-73.847201,Baychester Avenue Food Truck,40.892293,-73.843230,Food Truck
8,8,Wakefield,40.894705,-73.847201,Koss Quick Wash,40.891147,-73.850230,Laundromat
9,9,Co-op City,40.874294,-73.829939,Dollar Tree,40.870125,-73.828989,Discount Store


<h1>Hard work</h1>

In [47]:
%%time
cbg_df = pd.read_csv('cbg_patterns.csv') # big 245MB
print('file loaded')

file loaded
CPU times: user 6 s, sys: 244 ms, total: 6.24 s
Wall time: 6.21 s


In [48]:
print(cbg_df.shape)
print("Unique census_block_group: {}".format(len(cbg_df['census_block_group'].unique())))
pd.set_option('display.precision', 10)
cbg_df.dropna(inplace=True)
cbg_df['census_block_group'] = cbg_df['census_block_group'].astype(int)
cbg_df.head()

(220735, 13)
Unique census_block_group: 220735


,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day
0,10059508001,1538352000,1541030400,75122.0,18314.0,"{""010059501003"":127,""010059509001"":111,""010059...","{""010059501003"":109,""010810407002"":62,""0108104...",194724.0,"[""Chick-fil-A"",""mcdonalds"",""Marathon Petroleum...","[""walmart"",""mcdonalds"",""Dollar General"",""Chick...","[""CrossFit"",""Health Mart"",""Coldwell Banker""]","[2617,2457,2403,2519,2646,3007,3886,7566,5508,...","{""Monday"":12000,""Tuesday"":12224,""Wednesday"":10..."
1,10090506011,1538352000,1541030400,95649.0,38942.0,"{""010730113021"":210,""010090506022"":205,""010090...","{""010890111001"":271,""010730045001"":269,""010439...",120587.0,"[""Shell Oil"",""mcdonalds"",""Chick-fil-A"",""Chevron""]","[""walmart"",""mcdonalds"",""Shell Oil"",""Chick-fil-...",[],"[6556,6325,6222,6355,6586,7350,8568,8099,7378,...","{""Monday"":12125,""Tuesday"":12984,""Wednesday"":12..."
2,10479572002,1538352000,1541030400,14009.0,3039.0,"{""010479567011"":67,""010479567021"":60}","{""010479567021"":52}",67774.0,"[""Dollar General""]","[""walmart"",""Dollar General"",""mcdonalds"",""Chevr...","[""Dollar General""]","[807,790,796,786,851,951,1134,1797,1355,1241,1...","{""Monday"":2314,""Tuesday"":2340,""Wednesday"":2195..."
3,10690404001,1538352000,1541030400,128169.0,25418.0,"{""010690402013"":370,""010690402011"":322,""010690...","{""010690402024"":313,""010690415004"":203,""010450...",42684.0,"[""Chick-fil-A"",""Sam's Club"",""Dollar General"",""...","[""walmart"",""Dollar General"",""mcdonalds"",""Marat...","[""Chick-fil-A"",""Sam's Club"",""Olive Garden"",""mc...","[2121,1828,1784,1704,1861,2373,3730,7497,7093,...","{""Monday"":21141,""Tuesday"":21143,""Wednesday"":17..."
4,10730112101,1538352000,1541030400,51453.0,9499.0,"{""010090507001"":183,""010730113021"":167,""010730...","{""010730045001"":140,""010730027001"":123,""010730...",18878.0,"[""Chevron"",""Daylight Donuts"",""walmart""]","[""walmart"",""Chevron"",""Dollar General"",""Shell O...","[""Chevron"",""CrossFit""]","[3804,3716,3686,3672,3735,4115,4855,5946,4526,...","{""Monday"":8402,""Tuesday"":8414,""Wednesday"":8550..."


In [49]:
print(cbg_df.shape)
print("Unique census_block_group: {}".format(len(cbg_df['census_block_group'].unique())))
cbg_df.info()

(220518, 13)
Unique census_block_group: 220518
<class 'pandas.core.frame.DataFrame'>
Int64Index: 220518 entries, 0 to 220628
Data columns (total 13 columns):
census_block_group          220518 non-null int64
date_range_start            220518 non-null int64
date_range_end              220518 non-null int64
raw_visit_count             220518 non-null float64
raw_visitor_count           220518 non-null float64
visitor_home_cbgs           220518 non-null object
visitor_work_cbgs           220518 non-null object
distance_from_home          220518 non-null float64
related_same_day_brand      220518 non-null object
related_same_month_brand    220518 non-null object
top_brands                  220518 non-null object
popularity_by_hour          220518 non-null object
popularity_by_day           220518 non-null object
dtypes: float64(3), int64(3), object(7)
memory usage: 23.6+ MB


In [50]:
# geo data, change 'census_block_group' type to int64
dtype = {'census_block_group': 'int64', 'amount_land': 'float', 'amount_water': 'float', 'latitude': 'float', 'longitude': 'float'}
cbg_geo_df = pd.read_csv('safegraph_open_census_data/metadata/cbg_geographic_data.csv', dtype=dtype)
print('Geo is here')

Geo is here


In [ ]:
cbg_geo_df.head()

In [51]:
cbg_geo_df.set_index('census_block_group')
# cbg_df = cbg_df.join(cbg_geo_df, on='census_block_group')
cbg_df = cbg_df.merge(cbg_geo_df)


In [52]:
cbg_df.head()

,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day,amount_land,amount_water,latitude,longitude
0,10059508001,1538352000,1541030400,75122.0,18314.0,"{""010059501003"":127,""010059509001"":111,""010059...","{""010059501003"":109,""010810407002"":62,""0108104...",194724.0,"[""Chick-fil-A"",""mcdonalds"",""Marathon Petroleum...","[""walmart"",""mcdonalds"",""Dollar General"",""Chick...","[""CrossFit"",""Health Mart"",""Coldwell Banker""]","[2617,2457,2403,2519,2646,3007,3886,7566,5508,...","{""Monday"":12000,""Tuesday"":12224,""Wednesday"":10...",9139838.0,380622.0,31.9204100,-85.1757409
1,10090506011,1538352000,1541030400,95649.0,38942.0,"{""010730113021"":210,""010090506022"":205,""010090...","{""010890111001"":271,""010730045001"":269,""010439...",120587.0,"[""Shell Oil"",""mcdonalds"",""Chick-fil-A"",""Chevron""]","[""walmart"",""mcdonalds"",""Shell Oil"",""Chick-fil-...",[],"[6556,6325,6222,6355,6586,7350,8568,8099,7378,...","{""Monday"":12125,""Tuesday"":12984,""Wednesday"":12...",66284778.0,345142.0,33.8638835,-86.9035142
2,10479572002,1538352000,1541030400,14009.0,3039.0,"{""010479567011"":67,""010479567021"":60}","{""010479567021"":52}",67774.0,"[""Dollar General""]","[""walmart"",""Dollar General"",""mcdonalds"",""Chevr...","[""Dollar General""]","[807,790,796,786,851,951,1134,1797,1355,1241,1...","{""Monday"":2314,""Tuesday"":2340,""Wednesday"":2195...",16723739.0,517171.0,32.3573365,-87.0140035
3,10690404001,1538352000,1541030400,128169.0,25418.0,"{""010690402013"":370,""010690402011"":322,""010690...","{""010690402024"":313,""010690415004"":203,""010450...",42684.0,"[""Chick-fil-A"",""Sam's Club"",""Dollar General"",""...","[""walmart"",""Dollar General"",""mcdonalds"",""Marat...","[""Chick-fil-A"",""Sam's Club"",""Olive Garden"",""mc...","[2121,1828,1784,1704,1861,2373,3730,7497,7093,...","{""Monday"":21141,""Tuesday"":21143,""Wednesday"":17...",1835527.0,0.0,31.2373273,-85.4263314
4,10730112101,1538352000,1541030400,51453.0,9499.0,"{""010090507001"":183,""010730113021"":167,""010730...","{""010730045001"":140,""010730027001"":123,""010730...",18878.0,"[""Chevron"",""Daylight Donuts"",""walmart""]","[""walmart"",""Chevron"",""Dollar General"",""Shell O...","[""Chevron"",""CrossFit""]","[3804,3716,3686,3672,3735,4115,4855,5946,4526,...","{""Monday"":8402,""Tuesday"":8414,""Wednesday"":8550...",3010420.0,0.0,33.6717999,-86.6906102


In [53]:
%%time
from pandarallel import pandarallel
pandarallel.initialize()

def count_visitors(it):
    visitors = json.loads(it)
    return  len(visitors)

cbg_df['visitors_count'] = cbg_df['visitor_home_cbgs'].parallel_apply(count_visitors)
# cbg_df['visitors_count'] = cbg_df['visitor_home_cbgs'].apply(count_visitors)
# admit only visitor, with visit!
cbg_df = cbg_df[cbg_df['visitors_count'] != 0]
cbg_df.tail(20)

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 8 workers
CPU times: user 628 ms, sys: 468 ms, total: 1.1 s
Wall time: 3.7 s


,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day,amount_land,amount_water,latitude,longitude,visitors_count
220148,484530019102,1538352000,1541030400,113521.0,20678.0,"{""484530017601"":493,""484530017602"":351,""484530...","{""484530019182"":490,""484530019121"":268,""484530...",17497.0,"[""H-E-B""]","[""H-E-B"",""Shell Oil"",""starbucks"",""Whataburger""...","[""Rudy's Texas Bar-B-Q"",""Domino's Pizza"",""Shel...","[1972,1603,1539,1528,1669,2080,4117,6846,11457...","{""Monday"":17365,""Tuesday"":18355,""Wednesday"":18...",1558607.0,0.0,30.2676381,-97.8127563,56
220149,490351129202,1538352000,1541030400,62193.0,15197.0,"{""490351130201"":327,""490351130191"":274,""490351...","{""490351145003"":318,""490351128171"":163,""490351...",8728.0,"[""starbucks"",""Fred Meyer Jewelers""]","[""walmart"",""Smith's Food & Drug Stores"",""mcdon...","[""Fred Meyer Jewelers"",""starbucks"",""CUTERA"",""G...","[2521,2362,2302,2321,2382,2774,3468,5179,4873,...","{""Monday"":10383,""Tuesday"":10838,""Wednesday"":10...",1898876.0,0.0,40.5958382,-111.9460394,49
220151,510411005054,1538352000,1541030400,12923.0,2296.0,"{""510411005071"":150,""510411005062"":121,""510411...","{""510411004091"":76}",7496.0,"[""Wawa"",""Food Lion"",""Chick-fil-A""]","[""Wawa"",""Food Lion"",""walmart"",""Chick-fil-A"",""m...",[],"[1342,1326,1340,1360,1358,1360,1603,1876,1580,...","{""Monday"":2159,""Tuesday"":2135,""Wednesday"":2130...",595050.0,0.0,37.3452067,-77.4417022,5
220152,518100462114,1538352000,1541030400,29724.0,7495.0,"{""518100462172"":169,""518100462062"":98,""5181004...","{""515500208051"":98,""518100460064"":93,""51710000...",6971.0,"[""7-Eleven US"",""Wawa"",""Chick-fil-A""]","[""7-Eleven US"",""walmart"",""Wawa"",""Chick-fil-A"",...",[],"[1113,1055,1003,968,1026,1192,1769,2146,2339,1...","{""Monday"":4700,""Tuesday"":5012,""Wednesday"":4837...",569877.0,0.0,36.8034229,-76.1689409,12
220153,530050115035,1538352000,1541030400,31617.0,6543.0,"{""530050115042"":106,""530050115031"":91,""5300501...","{""530050108132"":155,""530050109014"":80,""5302102...",9528.0,"[""walmart"",""The Home Depot"",""mcdonalds""]","[""walmart"",""mcdonalds"",""Goodwill Industries"",""...","[""The Home Depot"",""Walgreens"",""Petco"",""starbuc...","[1025,986,945,931,1014,1172,1562,2605,2301,233...","{""Monday"":4794,""Tuesday"":4941,""Wednesday"":4865...",982964.0,0.0,46.1883424,-119.1651266,12
220155,530330035001,1538352000,1541030400,29971.0,9412.0,"{""530330004011"":54}","{""530330072001"":123,""530330081002"":120,""530330...",12244.0,"[""starbucks""]","[""starbucks"",""Safeway"",""Costco Wholesale Corp....","[""starbucks""]","[849,768,755,719,735,926,1257,2466,2664,2304,2...","{""Monday"":4674,""Tuesday"":4703,""Wednesday"":4762...",348426.0,0.0,47.6778488,-122.3516058,1
220156,530330085003,1538352000,1541030400,151838.0,54137.0,"{""530330081002"":117,""530330086002"":90,""5303303...","{""530330081002"":446,""530330053023"":349,""530330...",27622.0,"[""starbucks""]","[""starbucks"",""Safeway"",""Costco Wholesale Corp....",[],"[3574,2940,2646,2567,3006,4463,7828,10737,1119...","{""Monday"":25085,""Tuesday"":24404,""Wednesday"":23...",143795.0,0.0,47.6032597,-122.3255061,16
220157,530330227023,1538352000,1541030400,16046.0,2927.0,"{""530330220063"":66}","{""530330225004"":55,""530330228033"":52}",8965.0,"[""Gold's Gym"",""Ross Stores"",""Target"",""Baskin R...","[""starbucks"",""Costco Wholesale Corp."",""Safeway...",[],"[1014,964,949,938,959,1031,1221,1949,1708,1605...","{""Monday"":2853,""Tuesday"":2984,""Wednesday"":2665...",821935.0,0.0,47.6542335,-122.1982567,1
220158,530530704033,1538352000,1541030400,47666.0,5088.0,"{""530530704041"":168,""530530704042"":148,""530530...","{""530530704041"":70,""530330292061"":50}",10567.0,"[""starbucks"",""O'Reilly Auto Parts"",""mcdonalds""]","[""s

In [94]:
cbg_min_df = cbg_df[['census_block_group', 'latitude', 'longitude']]
cbg_min_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192299 entries, 0 to 220170
Data columns (total 3 columns):
census_block_group    192299 non-null int64
latitude              192299 non-null float64
longitude             192299 non-null float64
dtypes: float64(2), int64(1)
memory usage: 5.9 MB


In [95]:
dtype = {'state': 'str', 'state_fips': 'str', 'county_fips': 'str', 'county': 'str', 'class_code': 'str'}
fips_df = pd.read_csv('safegraph_open_census_data/metadata/cbg_fips_codes.csv', dtype=dtype)
fips_df['state_country_fips'] = fips_df['state_fips'] + fips_df['county_fips']
fips_df.drop(['state_fips','county_fips'], axis=1, inplace=True)


# fips_df[fips_df['state'] == 'NY'].head()

cbg_min_df['census_block_group'] = cbg_min_df['census_block_group'].astype(str, copy=False)
cbg_min_df['census_block_group'] = cbg_min_df['census_block_group'].str.slice(0,5)

cbg_min_df.head()




/home/wilton/jupyter/runtime/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/wilton/jupyter/runtime/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,census_block_group,latitude,longitude
0,10059,31.9204100,-85.1757409
1,10090,33.8638835,-86.9035142
2,10479,32.3573365,-87.0140035
3,10690,31.2373273,-85.4263314
4,10730,33.6717999,-86.6906102


In [111]:
# cbg_min_df = cbg_min_df.join(fips_df.set_index('state_country_fips'), on='census_block_group')

cbg_min_df.dropna(inplace=True)
cbg_min_df_nyc = cbg_min_df[cbg_min_df['state']=='NY']
cbg_min_df.shape

(157520, 6)

In [133]:
%%time
import math

merge_df = pd.DataFrame(columns=['lat', 'long', 'near_lat', 'near_long'])

def get_near_visitors(la, lg):
    nears = []
    # inputs
    radius = 25.0 # m - the following code is an approximation that stays reasonably accurate for distances < 100km
    jump = 2 # jump to center     
    centerLat = la # latitude of circle center, decimal degrees
    centerLon = lg # Longitude of circle center, decimal degrees

    # parameters
    N = 40 # number of discrete sample points to be generated along the circle

    # generate points
    circlePoints = []
    while radius > jump:
        for k in range(N):
            # compute
            angle = math.pi*2*k/N
            dx = radius*math.cos(angle)
            dy = radius*math.sin(angle)
            point = {}
            point['lat']=centerLat + (180/math.pi)*(dy/6378137)
            point['lon']=centerLon + (180/math.pi)*(dx/6378137)/math.cos(centerLat*math.pi/180)
            # add to list
            circlePoints.append(point)
        radius = radius-jump

    print("circlePoints: {}".format(len(circlePoints)))

    for c in circlePoints:
        pos = 2
        lat, lon = abs(c['lat']), abs(c['lon'])
        min_lat, min_lon = round(lat, pos), round(lon, pos)
#         print(min_lat, lat, min_lon, lon)
        tempa = cbg_min_df_nyc[(cbg_min_df_nyc['latitude'].abs().between(min_lat, lat)) & (cbg_min_df_nyc['longitude'].abs().between(min_lon, lon))]
        if tempa.size > 0:
#             print("Found ~ latitude {}, longitude {}".format(tempa['latitude'].iloc[0], tempa['longitude'].iloc[0]))
            nears.append({'lat': centerLat, 'long': centerLon, 'near_lat': tempa['latitude'].iloc[0], 'near_long': tempa['longitude'].iloc[0]})
            merge_df.append({'lat': centerLat, 'long': centerLon, 'near_lat': tempa['latitude'].iloc[0], 'near_long': tempa['longitude'].iloc[0]}, ignore_index=True)
            merge_df.to_csv('brutal.csv', mode='a', header=False)
        
            
    return nears


            
# Dyre Fish Market	40.8893176430	-73.8314525736	Seafood Restaurant
# get_near_visitors(40.8893176430, -73.8314525736)

# nyc_venues[nyc_venues['Neighborhood'].str.contains('Manhatt')].head()


    
# print(circlePoints)
# cbg_df[['latitude', 'longitude']].head()
# tempa = cbg_df[(cbg_df['latitude'] == 33.8638835) & (cbg_df['longitude'] == -86.9035142)]
# if tempa.size > 0:
#     print("Found ~ latitude {}, longitude {}".format(tempa['latitude'].iloc[0], tempa['longitude'].iloc[0]))
    
# pos = 5
# lat, lon = abs(33.8638835), abs(-86.9035142)
# min_lat, min_lon = round(lat, pos), round(lon, pos)
# print(min_lat, lat, min_lon, lon)
# tempa = cbg_df[(cbg_df['latitude'].abs().between(min_lat, lat)) & (cbg_df['longitude'].abs().between(min_lon, lon))]
# if tempa.size > 0:
#     print("Found ~ latitude {}, longitude {}".format(tempa['latitude'].iloc[0], tempa['longitude'].iloc[0]))    


CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 9.4 ms


In [135]:
%%time

from functools import partial
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

def doit(row):
    lat, lng = row['Venue Latitude'], row['Venue Longitude']
    print("looking for {}, {}".format(lat,lng))
#     for idx, ir in cbg_min_df_nyc.iterrows():
#         ila,ilo = ir['latitude'], ir['longitude']
#         km = distance.distance((lat,lng), (ila,ilo)).km
#         if km*1000 < 50:
#             print("near found **{} [{},{}] [{},{}]".format(km, lat,lng, ila,ilo))
        
    print(get_near_visitors(lat, lng))
#     merge_df = merge_df.append(get_near_visitors(lat, lng), ignore_index=True)

# nyc_venues[nyc_venues['Neighborhood'].str.contains('Bron')].head(1).apply(lambda row: demo(row['Venue Latitude'], row['Venue Longitude']), axis=1)
ok = nyc_venues[nyc_venues['Neighborhood'].str.contains('c')].head(200)
pool.map(doit, ok.T.to_dict().values())
pool.terminate()
pool.join()

# for idx, row in nyc_venues[nyc_venues['Neighborhood'].str.contains('Bron')].head(1).iterrows():
#     print(row['Neighborhood'], row['Venue Latitude'], row['Venue Longitude'])
#     get_near_visitors(row['Venue Latitude'], row['Venue Longitude'])

looking for 40.88601990111869, -73.82320747767669
looking for 40.81887820776004, -73.81679306899113
looking for 40.868814019292365, -73.83367127976227
looking for 40.88553926848355, -73.82915130799964
looking for 40.88798538407525, -73.83138309969671
looking for 40.863297869740656, -73.83556764832478
looking for 40.83609833012624, -73.85494781241097
looking for 40.863089, -73.833352
circlePoints: 480
circlePoints: 480
circlePoints: 480
circlePoints: 480
circlePoints: 480
circlePoints: 480
circlePoints: 480
circlePoints: 480
[]
looking for 40.835878819918506, -73.8556310850081
circlePoints: 480
[]
looking for 40.88563631373264, -73.82683955509486
circlePoints: 480
[]
looking for 40.81946172710321, -73.81735175518574
circlePoints: 480
[]
looking for 40.863099, -73.833831
circlePoints: 480
[]
looking for 40.888433208510854, -73.83127682668768
circlePoints: 480
[]
looking for 40.86780008007165, -73.83336456776759
[]
looking for 40.88633207186384, -73.8276156192422
circlePoints: 480
circleP

looking for 40.86915511893992, -73.83156599525223
circlePoints: 480
[]
looking for 40.83611083931938, -73.85490233193293
circlePoints: 480
[]
looking for 40.819222257989786, -73.81760066505247
circlePoints: 480
[]
looking for 40.889317642958616, -73.83145257358295
circlePoints: 480
[]
looking for 40.86811787434579, -73.83059312471944
circlePoints: 480
[]
looking for 40.88816692651785, -73.83152504085197
circlePoints: 480
[]
looking for 40.888251933965, -73.83145707477787
circlePoints: 480
[]
looking for 40.8686218839239, -73.83380771265148
circlePoints: 480
[]
looking for 40.86320122, -73.83454249
circlePoints: 480
[]
looking for 40.8187212325391, -73.81723942127736
circlePoints: 480
[]
looking for 40.83489492075461, -73.85535140525981
circlePoints: 480
[]
looking for 40.86322348484106, -73.83466718602823
circlePoints: 480
[]
looking for 40.88580984751669, -73.82805491348806
circlePoints: 480
[]
looking for 40.885384192918174, -73.8280987591753
circlePoints: 480
[]
looking for 40.88710

looking for 40.86631, -73.83032
circlePoints: 480
[]
looking for 40.88823525219036, -73.83128232689451
circlePoints: 480
[]
looking for 40.818949271515216, -73.8173667230774
circlePoints: 480
[]
looking for 40.83788039213797, -73.86013179887586
circlePoints: 480
[]
looking for 40.88461356066747, -73.83152944427044
circlePoints: 480
[]
looking for 40.8879034381361, -73.83125388185631
circlePoints: 480
[{'lat': 40.86322348484106, 'long': -73.83466718602823, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.86322348484106, 'long': -73.83466718602823, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.86322348484106, 'long': -73.83466718602823, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.86322348484106, 'long': -73.83466718602823, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.86322348484106, 'long': -73.83466718602823, 'near_lat': 40.861718700000004, 'near_long': -73.8344830

looking for 40.819391233745925, -73.81729767562672
circlePoints: 480
[{'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.864383462901294, 'long': -73.8346077587142, 'near_lat': 40.8617187000000

looking for 40.86862220285062, -73.83405221251074
circlePoints: 480
[]
looking for 40.86321156516152, -73.834300472422
circlePoints: 480
[]
looking for 40.888543185027366, -73.83127757319923
circlePoints: 480
[]
looking for 40.83681460824893, -73.86086299073949
circlePoints: 480
[]
looking for 40.865843040869976, -73.83770669702484
circlePoints: 480
[]
looking for 40.8193374870703, -73.81751013942011
circlePoints: 480
[]
looking for 40.88800860085501, -73.83133279875986
circlePoints: 480
[]
looking for 40.819204436132736, -73.81746681637895
circlePoints: 480
[]
looking for 40.868600575013495, -73.8312082077379
circlePoints: 480
[{'lat': 40.86321156516152, 'long': -73.834300472422, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.86321156516152, 'long': -73.834300472422, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat': 40.86321156516152, 'long': -73.834300472422, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, {'lat'

...

looking for 40.83524726742855, -73.9208570761204
circlePoints: 480
[]
looking for 40.82664104244332, -73.82248599450232
circlePoints: 480
[]
looking for 40.82859443171689, -73.8245923836983
circlePoints: 480
[]
looking for 40.83057397130016, -73.82646337151526
circlePoints: 480
[]
[{'lat': 40.83130668787119, 'long': -73.91994939928179, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.83130668787119, 'long': -73.91994939928179, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.83130668787119, 'long': -73.91994939928179, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.83130668787119, 'long': -73.91994939928179, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.83130668787119, 'long': -73.91994939928179, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.83130668787119, 'long': -73.91994939928179, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.83130668787119, 'long': -73.

looking for 40.834333011707436, -73.91806275125533
looking for 40.83336858139032, -73.91445285669495
circlePoints: 480
circlePoints: 480
[]
looking for 40.83160437411904, -73.91082828896594
circlePoints: 480
[]
looking for 40.57698137121013, -73.96421936506339
circlePoints: 480
[]
looking for 40.82595968425058, -73.82135166456301
circlePoints: 480
[]
looking for 40.8354434, -73.9209927
circlePoints: 480
[]
looking for 40.829223327529824, -73.82690605886773
circlePoints: 480
[]
looking for 40.826738890767004, -73.82248089184401
circlePoints: 480
[]
looking for 40.83639867889737, -73.91508827790081
circlePoints: 480
[]
looking for 40.831307, -73.920028
circlePoints: 480
[{'lat': 40.83336858139032, 'long': -73.91445285669495, 'near_lat': 40.8323781, 'near_long': -73.91278050000001}, {'lat': 40.83336858139032, 'long': -73.91445285669495, 'near_lat': 40.8323781, 'near_long': -73.91278050000001}, {'lat': 40.83336858139032, 'long': -73.91445285669495, 'near_lat': 40.8323781, 'near_long': -73.

looking for 40.83393302779996, -73.91667401417776
circlePoints: 480
[]
looking for 40.576748189880746, -73.96599329968467
circlePoints: 480
[]
looking for 40.8263461393095, -73.82217273497031
circlePoints: 480
[]
looking for 40.835872840412605, -73.92099072040764
circlePoints: 480
[]
[]
looking for 40.82629856858011, -73.82200190847016
looking for 40.826084215490816, -73.82150886877642
circlePoints: 480
circlePoints: 480
[]
looking for 40.832707863993534, -73.91779130262806
circlePoints: 480
[]
looking for 40.833523427558106, -73.9148243599908
circlePoints: 480
[]
looking for 40.577451780074064, -73.9614209929464
circlePoints: 480
[{'lat': 40.831307, 'long': -73.920028, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.831307, 'long': -73.920028, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.831307, 'long': -73.920028, 'near_lat': 40.8300026, 'near_long': -73.92006059999999}, {'lat': 40.831307, 'long': -73.920028, 'near_lat': 40.8300026, 'near_

looking for 40.57662179440902, -73.96639525809995
circlePoints: 480
[]
looking for 40.82641164206352, -73.8216045722783
circlePoints: 480
[]
looking for 40.83413508990203, -73.91689720106031
circlePoints: 480
[]
looking for 40.8309841, -73.8269655
circlePoints: 480
[]
looking for 40.82620533583085, -73.82123373829472
[]
circlePoints: 480
looking for 40.8347433098082, -73.91960077901727
circlePoints: 480
[]
looking for 40.57757524231774, -73.96344895612155
circlePoints: 480
[]
looking for 40.57769027608543, -73.96158037955199
circlePoints: 480
[]
looking for 40.577042186562295, -73.96444682680435
circlePoints: 480
[]
looking for 40.83427075692771, -73.91685080669906
circlePoints: 480
[]
looking for 40.576379530336716, -73.96579665839249
circlePoints: 480
[]
looking for 40.832606, -73.919087
circlePoints: 480
[{'lat': 40.833523427558106, 'long': -73.9148243599908, 'near_lat': 40.8323781, 'near_long': -73.91278050000001}, {'lat': 40.833523427558106, 'long': -73.9148243599908, 'near_lat': 

looking for 40.830929582269505, -73.91662582679406
circlePoints: 480
[]
looking for 40.57618, -73.967247
circlePoints: 480
[]
looking for 40.57708462497956, -73.96452675359363
circlePoints: 480
[]
looking for 40.57471289946765, -73.96980531547729
circlePoints: 480
[]
looking for 40.575470783909594, -73.96910308656875
circlePoints: 480
[]
[]
looking for 40.57699238667643, -73.96487048242537
circlePoints: 480
[]
[]
looking for 40.83530662693773, -73.92072176482904
circlePoints: 480
[]
looking for 40.575391107735655, -73.96500435141279
circlePoints: 480
[]
looking for 40.57784143, -73.96120404
circlePoints: 480
[]
looking for 40.575517505899995, -73.96237246820378
circlePoints: 480
[]
looking for 40.57499280167667, -73.96275752289714
circlePoints: 480
[]
[]
looking for 40.57874555670518, -73.95977893543797
circlePoints: 480
[]
looking for 40.577605575374506, -73.96106953804406
circlePoints: 480
[]
looking for 40.5775791, -73.9607877
circlePoints: 480
[]
looking for 40.57659383540472, -73.

In [120]:
# merge_df = merge_df.append({'lat': 40.86320122, 'long': -73.83454249, 'near_lat': 40.861718700000004, 'near_long': -73.83448309999999}, ignore_index=True)
merge_df.head()

,lat,long,near_lat,near_long


In [ ]:
lat, lng = 40.9096868, -73.8798174
ila,ilo = 40.9105384, -73.8531227
km = distance.distance((lat,lng), (ila,ilo)).km
if km*1000 < 3000:
    print("near found ({}) [{},{}] [{},{}]".format(km, lat,lng, ila,ilo))

In [ ]:
nyc_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(nyc_venues['Venue Category'].unique())))

In [ ]:
%%time

# Analyze Each Neighborhood --------------
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

nyc_onehot.drop('Neighborhood', axis=1, inplace=True)
# with method DataFrame.insert may resume some task.
nyc_onehot.insert(0, 'Neighborhood', nyc_venues['Neighborhood'])
nyc_onehot.head()


In [ ]:
nyc_onehot.shape

In [ ]:
%%time

nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
print(nyc_grouped.shape)
nyc_grouped.head()

In [ ]:
1

In [ ]:
%%time
num_top_venues = 5

for hood in nyc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nyc_grouped[nyc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head(10)

<h1>Cluster Neighborhoods</h1>

In [ ]:
# set number of clusters
kclusters = 6

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
nyc_merged = neighborhoods
print("nyc_merged/neighborhoods  {}".format(neighborhoods.shape))
print("neighborhoods_venues_sorted {}".format(neighborhoods_venues_sorted.shape))
    
# join convert some infinite kmeans.labels_ bad bad ....
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print("before isfinite {}".format(nyc_merged.shape))
# # nyc_merged['Cluster Labels'].dropna(inplace=True)

for n,h in enumerate(np.isfinite(nyc_merged['Cluster Labels'])):
    if not h:
        print(nyc_merged.values[n])

# clear infinite labels after join         
nyc_merged = nyc_merged[np.isfinite(nyc_merged['Cluster Labels'])]
print("after... isfinite {}".format(nyc_merged.shape))

nyc_merged['Cluster Labels'].dropna(inplace=True)
print("drop NaN {}".format(nyc_merged.shape))
nyc_merged['Cluster Labels'] = nyc_merged['Cluster Labels'].astype(int)

nyc_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 3, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]


In [ ]:
1